In [37]:
%matplotlib qt4

import hyperspy.api as hs

import numpy as np
import matplotlib.pyplot as plt
from scipy import io as io
import astropy.units as u

Using matplotlib backend: MacOSX


In [3]:
hs?

In [4]:
hs.load?

In [5]:
hs.signals?

In [6]:
file = './data_v2/camera01/pro2016_iquv0000.sav'

In [9]:
pro2016_1 = io.readsav(file, python_dict = True, verbose = False )
iquv = pro2016_1['iquv']
iquv.shape

(39, 4, 512, 256)

***
### 'loading' data from a numpy array

- to create 2D signal using the last 2 axis of a numpy.ndarray
> s = hs.signals.Signal2D(numpy.ndarray)

- to create 1D signal using the last 1 axis of a numpy.ndarray
> s = hs.signals.Signal1D(numpy.ndarray)

In [11]:
s = hs.signals.Signal2D(iquv)
s

<Signal2D, title: , dimensions: (4, 39|256, 512)>

***
### setting axis roperties using `axes_manager`

In [12]:
s.axes_manager

<Axes manager, axes: (4, 39|256, 512)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
     <undefined> |      4 |      0 |       0 |       1 | <undefined> 
     <undefined> |     39 |      0 |       0 |       1 | <undefined> 
---------------- | ------ | ------ | ------- | ------- | ------ 
     <undefined> |    256 |        |       0 |       1 | <undefined> 
     <undefined> |    512 |        |       0 |       1 | <undefined>

In [18]:
s.axes_manager[3]

<Unnamed 3rd axis, size: 512>

In [22]:
s.axes_manager[0].name = 'stokes'
s.axes_manager[1].name = 'scan'
s.axes_manager[2].name = 'slit'
s.axes_manager[3].name = 'wavelength'

In [23]:
s.axes_manager

<Axes manager, axes: (4, 39|256, 512)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
          stokes |      4 |      0 |       0 |       1 | <undefined> 
            scan |     39 |      0 |       0 |       1 | <undefined> 
---------------- | ------ | ------ | ------- | ------- | ------ 
            slit |    256 |        |       0 |       1 | <undefined> 
      wavelength |    512 |        |       0 |       1 | <undefined>

#### once the name of an axis has been defined it is possible to request it by its name 

In [24]:
s.axes_manager['stokes']

<stokes axis, size: 4, index: 0>

In [33]:
s.axes_manager['scan'].units = 'arcsec'
s.axes_manager['scan'].scale = 2
s.axes_manager['scan'].offset = 16*60 + 20

In [34]:
s.axes_manager

<Axes manager, axes: (4, 39|256, 512)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
          stokes |      4 |      0 |       0 |       1 | <undefined> 
            scan |     39 |      0 | 9.8e+02 |       2 | arcsec 
---------------- | ------ | ------ | ------- | ------- | ------ 
            slit |    256 |        |       0 |       1 | <undefined> 
      wavelength |    512 |        |       0 |       1 | <undefined>

***
### saving and loading files

In [35]:
s.save('./pro2016_1.hdf5')

In [39]:
hs.set_log_level('INFO')

In [43]:
s = hs.load('./pro2016_1.hdf5')

***
### transforming between signal subclasses

In [45]:
s

<Signal2D, title: , dimensions: (4, 39|256, 512)>

In [46]:
s.metadata

├── General
│   └── title = 
└── Signal
    ├── binned = False
    └── signal_type = 

In [53]:
profile = s.as_signal1D('wavelength')
profile

<Signal1D, title: , dimensions: (256, 4, 39|512)>

In [55]:
stokes_profile = s.as_signal2D(('stokes','wavelength'))
stokes_profile

<Signal2D, title: , dimensions: (256, 39|4, 512)>

***
### `.isig` and `.inav` allow us to look deep inside signal space and navigation space by indexing

In [59]:
profile

<Signal1D, title: , dimensions: (256, 4, 39|512)>

In [58]:
profile.isig[0]

<BaseSignal, title: , dimensions: (256, 4, 39|)>

In [60]:
profile.inav[100,1,30]

<Signal1D, title: , dimensions: (|512)>

In [64]:
profile.inav[100,1,30].data

array([  1.82827515e+02,   2.48957764e+02,   2.57711670e+02,
         8.31903152e+01,   7.71073837e+01,   2.58094635e+02,
         4.88649063e+01,   2.34362854e+02,   2.10187241e+02,
         1.37573563e+02,   1.98837265e+02,   2.81870575e+02,
         1.01631714e+02,  -2.07013702e+01,   5.23385925e+01,
        -9.93670883e+01,  -6.05286942e+01,  -6.03402214e+01,
         2.20647163e+01,  -7.28950577e+01,  -8.03648853e+00,
         3.35528450e+01,   1.85867443e+01,   1.32416809e+02,
        -1.33624229e+01,   8.12513351e+01,  -1.13630495e+01,
         1.57993793e+00,   1.52132111e+02,  -3.67137604e+01,
         1.66010162e+02,  -1.90122490e+01,  -7.63213577e+01,
         1.39563614e+02,   8.40409241e+01,   3.25370522e+01,
         1.47256760e+02,   5.31235428e+01,   9.39904861e+01,
         1.54366714e+02,   3.52943764e+01,  -4.33570938e+01,
         7.20513535e+01,   4.87029762e+01,   1.84720097e+01,
         4.63365059e+01,   1.25275177e+02,  -1.35724289e+02,
        -1.01941116e+02,

#### Unlike numpy, HyperSpy supports indexing using decimal numbers, in which case HyperSpy indexes using the axis scales instead of the indices.

In [65]:
s.axes_manager['scan'].scale

2.0

In [69]:
s.axes_manager['scan'].axis

array([  980.,   982.,   984.,   986.,   988.,   990.,   992.,   994.,
         996.,   998.,  1000.,  1002.,  1004.,  1006.,  1008.,  1010.,
        1012.,  1014.,  1016.,  1018.,  1020.,  1022.,  1024.,  1026.,
        1028.,  1030.,  1032.,  1034.,  1036.,  1038.,  1040.,  1042.,
        1044.,  1046.,  1048.,  1050.,  1052.,  1054.,  1056.])

In [73]:
(profile.inav[100,1,30].data == profile.inav[100,1,1040.0].data).all()

True

In [75]:
profile.inav[100,1,30].axes_manager

Signal axis name,size,offset,scale,units
wavelength,512,0.0,1.0,



#### operations are performed element-wise. When the dimensions of the signals are not equal numpy broadcasting rules apply independently for the navigation and signal axes.

***
### summary statistics of the signal

In [76]:
profile.print_summary_statistics()

Summary statistics
------------------
mean:	14048.452
std:	70125.164

min:	-89064.828
Q1:	38.998
median:	230.065
Q3:	975.510
max:	1927423.375


In [78]:
profile.inav[100,1,30].print_summary_statistics()

Summary statistics
------------------
mean:	30.218
std:	80.983

min:	-202.294
Q1:	-29.897
median:	30.587
Q3:	82.220
max:	289.509


In [81]:
[profile.inav[100,i,30].print_summary_statistics() for i in range(4)]

Summary statistics
------------------
mean:	19884.787
std:	2596.107

min:	2760.682
Q1:	19440.454
median:	20480.051
Q3:	21275.644
max:	24158.172
Summary statistics
------------------
mean:	30.218
std:	80.983

min:	-202.294
Q1:	-29.897
median:	30.587
Q3:	82.220
max:	289.509
Summary statistics
------------------
mean:	67.344
std:	95.750

min:	-211.911
Q1:	3.780
median:	69.903
Q3:	138.413
max:	288.391
Summary statistics
------------------
mean:	401.386
std:	98.988

min:	53.241
Q1:	332.586
median:	401.073
Q3:	470.039
max:	652.561


[None, None, None, None]

In [82]:
hs.plot.plot_histograms(profile.inav[100,:,30], legend='auto')

/Users/kouui/anaconda/lib/python3.5/site-packages/matplotlib/__init__.py:898: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [89]:
hs.plot.plot_spectra(profile.inav[100,0,20], legend='auto')

/Users/kouui/anaconda/lib/python3.5/site-packages/matplotlib/__init__.py:898: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [90]:
hs.plot.plot_spectra(profile.inav[100,1:4,20], legend='auto')

/Users/kouui/anaconda/lib/python3.5/site-packages/matplotlib/__init__.py:898: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


***
### etc

In [93]:
s.axes_manager

<Axes manager, axes: (4, 39|256, 512)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
          stokes |      4 |      0 |       0 |       1 | <undefined> 
            scan |     39 |      0 | 9.8e+02 |       2 | arcsec 
---------------- | ------ | ------ | ------- | ------- | ------ 
            slit |    256 |        |       0 |       1 | <undefined> 
      wavelength |    512 |        |       0 |       1 | <undefined>

In [100]:
hs.plot.plot_images(s.inav[:,2])

/Users/kouui/anaconda/lib/python3.5/site-packages/hyperspy/drawing/utils.py:855: UserWarning: Axes labels were requested, but one or both of the axes units and/or name are undefined. Axes decorations have been set to 'ticks' instead.
  'Axes labels were requested, but one '
